In [1]:
import pandas as pd
import numpy as np
from scipy.stats import sem, t
import matplotlib.pyplot as plt


In [2]:
import numpy as np
from scipy.signal import resample

def downsample_to_targetrate(original_data, eyetracking_rate, target_rate):
    """
    Downsample eye-tracking data to a specified target rate.

    Args:
        original_data (np.array): Eye-tracking data array with columns for timestamp, X, Y, and pupil size.
        eyetracking_rate (int): Sampling rate of the original data.
        target_rate (float): Desired target sampling rate.

    Returns:
        np.array: Downsampled eye-tracking data.
    """
    # Remove NaNs
    def remove_nans(data):
        for col in range(data.shape[1]):
            nan_indices = np.isnan(data[:, col])
            if np.any(nan_indices):
                data[nan_indices, col] = np.interp(
                    np.flatnonzero(nan_indices),
                    np.flatnonzero(~nan_indices),
                    data[~nan_indices, col]
                )
        return data

    original_data = remove_nans(original_data)

    # Calculate total number of data points in target rate
    eyetracking_in_sec = len(original_data) / eyetracking_rate
    total_target_points = int(eyetracking_in_sec * target_rate)

    # Resample data columns
    downsampled_t = np.linspace(
        original_data[0, 0], original_data[-1, 0], total_target_points
    )
    downsampled_x = resample(original_data[:, 1], total_target_points)
    downsampled_y = resample(original_data[:, 2], total_target_points)

    # Stack into final array
    downsampled_data = np.column_stack((downsampled_t, downsampled_x, downsampled_y))

    return downsampled_data



In [3]:
import pandas as pd
import numpy as np
def adapt_evaluation(participant_evaluation):
    pred_y = participant_evaluation["pred_y"]
    pred_y_median = np.nanmedian(pred_y, axis=1)
    pred_uncertainty = abs(participant_evaluation["euc_pred"])
    pred_uncertainty_median = np.nanmedian(pred_uncertainty, axis=1)
    df_pred_median = pd.DataFrame(
        np.concatenate(
            (pred_y_median, pred_uncertainty_median[..., np.newaxis]), axis=1),
        columns=["X", "Y", "Uncertainty"],
    )
    # With subTR
    subtr_values = np.concatenate((pred_y, pred_uncertainty[..., np.newaxis]),
                                  axis=2)
    index = pd.MultiIndex.from_product(
        [range(subtr_values.shape[0]),
         range(subtr_values.shape[1])],
        names=["TR", "subTR"])
    df_pred_subtr = pd.DataFrame(subtr_values.reshape(-1,
                                                      subtr_values.shape[-1]),
                                 index=index,
                                 columns=["X", "Y", "pred_error"])

    return df_pred_median, df_pred_subtr

In [4]:
def euclidean_distance(eye_data, pred_x, pred_y): 
     eucl_dist = np.sqrt(
        (eye_data[:int(len(pred_x)), 0] - pred_x[:int(len(eye_data))])**2 + (eye_data[:int(len(pred_y)), 1] - pred_y[:int(len(eye_data))])**2)

     return eucl_dist

In [5]:
import numpy as np

def chunk_and_median(eyetracking_data, sampling_rate=1000, chunk_duration=1.2):
    """
    Splits continuous eyetracking data into chunks of specified duration
    and computes the median for each chunk, ensuring no NaNs are returned.

    Parameters:
    - eyetracking_data: 1D NumPy array (continuous signal)
    - sampling_rate: int, samples per second (default: 1000 Hz)
    - chunk_duration: float, duration of each chunk in seconds (default: 1.2 s)

    Returns:
    - medians: 1D NumPy array with median values per chunk
    """
    # Remove NaNs from input
    eyetracking_data = np.nan_to_num(eyetracking_data, nan=0.0)

    chunk_size = int(sampling_rate * chunk_duration)
    num_chunks = len(eyetracking_data) // chunk_size

    medians = np.array([
        np.nanmedian(eyetracking_data[i * chunk_size: (i + 1) * chunk_size])  # Ensures NaN-safe median
        for i in range(num_chunks)
    ])

    return medians


In [6]:
import numpy as np
import matplotlib.pyplot as plt

def segment_and_filter_eyetracking(data, window_size=1200, min_valid_ratio=0.5, target_points=10):
    """
    Segments eyetracking data into 1.2s windows, filters windows with >50% valid data,
    and downsamples to the target number of points using nth-point selection. 
    If a window is invalid, returns NaNs.

    Args:
        data (numpy array): 1D array of eyetracking data (1000 Hz, NaNs for blinks).
        window_size (int): Number of samples per window (1200 for 1.2s at 1000Hz).
        min_valid_ratio (float): Minimum valid data ratio to keep a window (0.5 means at least 600 non-NaN).
        target_points (int): Number of points to downsample to per valid window.

    Returns:
        list: List of downsampled windows (each as a numpy array, either valid data or NaNs).
    """
    downsampled_windows = []
    step_size = window_size // target_points  # Step size for nth-point selection

    for start in range(0, len(data) - window_size + 1, window_size):  # Non-overlapping windows
        window = data[start:start + window_size]
        
        # Count non-NaN values
        valid_count = np.sum(~np.isnan(window))
        
        if valid_count >= window_size * min_valid_ratio:
            # Select every nth point (fixed intervals)
            downsampled_window = window[::step_size][:target_points]
        else:
            # Return a 10-element array of NaNs if window is invalid
            downsampled_window = np.full(target_points, np.nan)
        
        downsampled_windows.append(downsampled_window)
    
    return downsampled_window


In [8]:
import numpy as np
import matplotlib.pyplot as plt

def segment_and_filter_eyetracking(data, window_size=1200, min_valid_ratio=0.5, target_points=10):
    """
    Segments eyetracking data into 1.2s windows, filters windows with >50% valid data,
    and downsamples to the target number of points using mean pooling. If a window is invalid, returns NaNs.

    Args:
        data (numpy array): 1D array of eyetracking data (1000 Hz, NaNs for blinks).
        window_size (int): Number of samples per window (1200 for 1.2s at 1000Hz).
        min_valid_ratio (float): Minimum valid data ratio to keep a window (0.5 means at least 600 non-NaN).
        target_points (int): Number of points to downsample to per valid window.

    Returns:
        list: List of downsampled windows (each as a numpy array, either valid data or NaNs).
    """
    downsampled_windows = []
    bin_size = window_size // target_points  # Number of samples per bin

    for start in range(0, len(data) - window_size + 1, window_size):  # Non-overlapping windows
        window = data[start:start + window_size]
        
        # Count non-NaN values
        valid_count = np.sum(~np.isnan(window))
        
        if valid_count >= window_size * min_valid_ratio:
            # Mean pooling: Divide the window into `target_points` bins and take the mean of each
            downsampled_window = np.array([
                np.nanmean(window[i:i + bin_size]) for i in range(0, window_size, bin_size)
            ])
        else:
            # Return a 10-element array of NaNs if window is invalid
            downsampled_window = np.full(target_points, np.nan)
        
        downsampled_windows.append(downsampled_window)
    
    return downsampled_windows


In [8]:
subjects = ["sub-03","sub-04", "sub-05", "sub-06", "sub-07","sub-09","sub-11", "sub-13", "sub-14", "sub-15"]

In [9]:
import pandas as pd
from scipy.signal import resample

design_matrix = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/exp_design/sub-02/sub-02_ses-02_task-DeepMReyeClosed_run-01_design_matrix.tsv", sep ="\t")
task_1_trials = np.array(design_matrix["task_eyes_open_task"])
task_2_trials = np.array(design_matrix["task_no_stimulus_task"])
task_3_trials = np.array(design_matrix["task_eyes_blink_task"])


resampled_task_1_trials = resample(task_1_trials, 313)
resampled_task_2_trials = resample(task_2_trials, 313)
resampled_task_3_trials = resample(task_3_trials, 313)


task_1_trials_bool = resampled_task_1_trials > 0.5
task_2_trials_bool = resampled_task_2_trials > 0.5
task_3_trials_bool = resampled_task_3_trials > 0.5


print(len(task_1_trials_bool))

313


In [14]:
# Calculate EE FT vs Eyetracking and save
tasks = ["open", "no_stim", "blink", "all"]
runs = 3 

evaluation_ft = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed.npy", allow_pickle=True).item()

for subject in subjects: 
    for run in range(runs): 
         # load eye tracking 
            eye_data = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_run_0{run+1}_eyedata.tsv.gz", compression='gzip', delimiter='\t')
            eye_data = eye_data[['timestamps','x', 'y']].to_numpy()     
            # downsample eye data 
            eye_data_downsampled_x = chunk_and_median(eye_data[:,1])
            eye_data_downsampled_y = chunk_and_median(eye_data[:,2])
            eye_data_downsampled = np.stack((eye_data_downsampled_x, eye_data_downsampled_y), axis=1)

            for task in tasks: 
                subject_data_ft = evaluation_ft[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data/{subject}_DeepMReyeClosed_label.npz"]
                df_pred_median_ft, df_pred_subtr_ft = adapt_evaluation(subject_data_ft)

                if run == 0: 
                    # Fine Tuned
                    sub_run_X_ft = np.array(df_pred_median_ft['X'])[:313]
                    sub_run_Y_ft = np.array(df_pred_median_ft['Y'])[:313]
                    
                    
                    if task == "open":
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool] 
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_1_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_1_trials_bool] 

                        ee_ft = euclidean_distance(eye_data_downsampled_task_1, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_1 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task == "no_stim": 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 

                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_2_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_2_trials_bool] 
                        
                        ee_ft = euclidean_distance(eye_data_downsampled_task_2, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_1 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task == "blink":
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_3_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_3_trials_bool] 
                        
                        ee_ft = euclidean_distance(eye_data_downsampled_task_3, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_1 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    
                    elif task == "all":
                        eye_data_downsampled_task_all = eye_data_downsampled 

                        # Fine Tuned
                        ee_ft = euclidean_distance(eye_data_downsampled_task_all, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_1 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    

                
                elif run == 1: 
                    sub_run_X_ft = np.array(df_pred_median_ft['X'])[313:626]
                    sub_run_Y_ft = np.array(df_pred_median_ft['Y'])[313:626]
                    
                    if task == "open":
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool]
                        # Scaled
                        sub_run_X_ft = sub_run_X_ft[task_1_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_1_trials_bool] 
                        
                        ee_ft = euclidean_distance(eye_data_downsampled_task_1, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_2 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task == "no_stim": 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_2_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_2_trials_bool] 

                        ee_ft = euclidean_distance(eye_data_downsampled_task_2, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_2 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task == "blink":
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_3_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_3_trials_bool] 

                        ee_ft = euclidean_distance(eye_data_downsampled_task_3, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_2 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    
                    elif task == "all":
                        eye_data_downsampled_task_all = eye_data_downsampled 

                        # Fine Tuned
                        ee_ft = euclidean_distance(eye_data_downsampled_task_all, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_2 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                        
                        
                elif run == 2: 
                    sub_run_X_ft = np.array(df_pred_median_ft['X'])[626:]
                    sub_run_Y_ft = np.array(df_pred_median_ft['Y'])[626:]
                    

                    if task == "open":
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool] 
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_1_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_1_trials_bool] 

                        ee_ft = euclidean_distance(eye_data_downsampled_task_1, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_3 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                        
                
                    elif task == "no_stim": 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_2_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_2_trials_bool] 

                        ee_ft = euclidean_distance(eye_data_downsampled_task_2, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_3 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                        

                    elif task == "blink":
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_3_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_3_trials_bool] 
                        
                        ee_ft = euclidean_distance(eye_data_downsampled_task_3, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_3 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    
                    elif task == "all":
                        eye_data_downsampled_task_all = eye_data_downsampled 

                        # Fine Tuned
                        ee_ft = euclidean_distance(eye_data_downsampled_task_all, sub_run_X_ft, sub_run_Y_ft)
                        eucl_dist_df_3 = pd.DataFrame(ee_ft, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

Fine Tuned

In [10]:
# Calculate mean and 75th percentile of EE FT 
import pandas as pd
import numpy as np

# Fine Tuned EE 

# Define tasks
tasks = ["open", "no_stim", "blink", "all"]

# Initialize dictionary to store results
subject_ee_means_ft = {}
subject_ee_perc_ft = {}

for subject in subjects:
    subject_ee_means_ft[subject] = {}
    subject_ee_perc_ft[subject] = {}
    
    for task in tasks:
        # Load data for each run
        ee_run_01 = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_01_ee.tsv.gz", compression='gzip', delimiter='\t')[['ee']].to_numpy()
        ee_run_02 = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_02_ee.tsv.gz", compression='gzip', delimiter='\t')[['ee']].to_numpy()
        ee_run_03 = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_03_ee.tsv.gz", compression='gzip', delimiter='\t')[['ee']].to_numpy()
        
        # Compute mean across all runs
        all_ee = np.concatenate([ee_run_01, ee_run_02, ee_run_03])
        mean_ee = np.mean(all_ee)
        perc_ee = np.percentile(all_ee, 75)
        
        # Save in dictionaries
        subject_ee_means_ft[subject][task] = mean_ee
        subject_ee_perc_ft[subject][task] = perc_ee
        print(f"{task}, {subject}: {perc_ee}")


#mean of all runs 
#75th percentile over mean 


open, sub-03: 2.1090044437428315
no_stim, sub-03: 4.468617335804004
blink, sub-03: 5.2673418692980185
all, sub-03: 5.884148603616918
open, sub-04: 5.070741565371426
no_stim, sub-04: 5.366786818179806
blink, sub-04: 10.392180806094647
all, sub-04: 6.8899018998346495
open, sub-05: 4.50262909654114
no_stim, sub-05: 4.322924968361133
blink, sub-05: 7.174828787067316
all, sub-05: 7.75631020757946
open, sub-06: 2.1377654010023215
no_stim, sub-06: 2.910022861689622
blink, sub-06: 6.143729849858076
all, sub-06: 4.403062487358888
open, sub-07: 5.57947349422395
no_stim, sub-07: 6.358155921633766
blink, sub-07: 8.993528535902179
all, sub-07: 7.164623823563243
open, sub-09: 6.6696635948001965
no_stim, sub-09: 5.4638404210459415
blink, sub-09: 5.209568335543533
all, sub-09: 7.1614382737604405
open, sub-11: 9.654606753079314
no_stim, sub-11: 10.350957497811605
blink, sub-11: 9.553345738386607
all, sub-11: 10.191495596211826
open, sub-13: 4.730693244716654
no_stim, sub-13: 6.068538683373458
blink, su

In [ ]:
# Calculate EE Calib Pretrained vs Eyetracking and save
tasks = ["open", "no_stim", "blink", "all"]
runs = 3 

evaluation_pt = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed_calibpretrained.npy", allow_pickle=True).item()

for subject in subjects: 
    for run in range(runs): 
         # load eye tracking 
            eye_data = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_run_0{run+1}_eyedata.tsv.gz", compression='gzip', delimiter='\t')
            eye_data = eye_data[['timestamps','x', 'y']].to_numpy()     
            # downsample eye data 
            eye_data_downsampled_x = chunk_and_median(eye_data[:,1])
            eye_data_downsampled_y = chunk_and_median(eye_data[:,2])
            eye_data_downsampled = np.stack((eye_data_downsampled_x, eye_data_downsampled_y), axis=1)

            for task in tasks: 
                subject_data_pt = evaluation_pt[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data_pretrained/{subject}_DeepMReyeClosed_no_label.npz"]
                df_pred_median_pt, df_pred_subtr_pt = adapt_evaluation(subject_data_pt)

                if run == 0: 
                    # Pretrained
                    sub_run_X_pt = np.array(df_pred_median_pt['X'])[:313]
                    sub_run_Y_pt = np.array(df_pred_median_pt['Y'])[:313]
                    #sub_run_Y_pt = -1 * sub_run_Y_pt
                    
                    
                    if task == "open":
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool] 
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_1_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_1_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_1, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_1 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task == "no_stim": 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_2_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_2_trials_bool] 
                        
                        ee_pt = euclidean_distance(eye_data_downsampled_task_2, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_1 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task ==  "blink":
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # pretrained
                        sub_run_X_pt = sub_run_X_pt[task_3_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_3_trials_bool] 
                        
                        ee_pt = euclidean_distance(eye_data_downsampled_task_3, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_1 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    
                    elif task == "all":
                        eye_data_downsampled_task_all = eye_data_downsampled 

                        # Pretrained
                        ee_pt = euclidean_distance(eye_data_downsampled_task_all, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_1 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    

                
                elif run == 1: 
                    sub_run_X_pt = np.array(df_pred_median_pt['X'])[313:626]
                    sub_run_Y_pt = np.array(df_pred_median_pt['Y'])[313:626]
                    #sub_run_Y_pt = -1 * sub_run_Y_pt
                    
                    if task == "open":
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool]
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_1_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_1_trials_bool] 
                        
                        ee_pt = euclidean_distance(eye_data_downsampled_task_1, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_2 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task == "no_stim": 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_2_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_2_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_2, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_2 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task ==  "blink":
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_3_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_3_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_3, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_2 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    
                    elif task == "all":
                        eye_data_downsampled_task_all = eye_data_downsampled 

                        # Pretrained
                        ee_pt = euclidean_distance(eye_data_downsampled_task_all, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_2 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                        
                        
                elif run == 2: 
                    sub_run_X_pt = np.array(df_pred_median_pt['X'])[626:]
                    sub_run_Y_pt = np.array(df_pred_median_pt['Y'])[626:]
                    #sub_run_Y_pt = -1 * sub_run_Y_pt
                    

                    if task == "open":
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool] 
                        #Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_1_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_1_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_1, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_3 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                        
                
                    elif task == "no_stim": 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_2_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_2_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_2, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_3 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                        

                    elif task ==  "blink":
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_3_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_3_trials_bool] 
                        
                        ee_pt = euclidean_distance(eye_data_downsampled_task_3, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_3 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    
                    elif task == "all":
                        eye_data_downsampled_task_all = eye_data_downsampled 

                        # Pretrained
                        ee_pt = euclidean_distance(eye_data_downsampled_task_all, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_3 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_calibpretrained.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

In [ ]:
# Calculate EE original Pretrained vs Eyetracking and save
tasks = ["open", "no_stim", "blink", "all"]
runs = 3 

evaluation_pt_old = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed_pretrained.npy", allow_pickle=True).item()

for subject in subjects: 
    for run in range(runs): 
         # load eye tracking 
            eye_data = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_run_0{run+1}_eyedata.tsv.gz", compression='gzip', delimiter='\t')
            eye_data = eye_data[['timestamps','x', 'y']].to_numpy()     
            # downsample eye data 
            eye_data_downsampled_x = chunk_and_median(eye_data[:,1])
            eye_data_downsampled_y = chunk_and_median(eye_data[:,2])
            eye_data_downsampled = np.stack((eye_data_downsampled_x, eye_data_downsampled_y), axis=1)

            for task in tasks: 
                subject_data_pt = evaluation_pt_old[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data_pretrained/{subject}_DeepMReyeClosed_no_label.npz"]
                df_pred_median_pt, df_pred_subtr_pt = adapt_evaluation(subject_data_pt)

                if run == 0: 
                    # Pretrained
                    sub_run_X_pt = np.array(df_pred_median_pt['X'])[:313]
                    sub_run_Y_pt = np.array(df_pred_median_pt['Y'])[:313]
                    sub_run_Y_pt = -1 * sub_run_Y_pt
                    
                    
                    if task == "open":
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool] 
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_1_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_1_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_1, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_1 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task == "no_stim": 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_2_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_2_trials_bool] 
                        
                        ee_pt = euclidean_distance(eye_data_downsampled_task_2, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_1 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task ==  "blink":
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # pretrained
                        sub_run_X_pt = sub_run_X_pt[task_3_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_3_trials_bool] 
                        
                        ee_pt = euclidean_distance(eye_data_downsampled_task_3, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_1 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    
                    elif task == "all":
                        eye_data_downsampled_task_all = eye_data_downsampled 

                        # Pretrained
                        ee_pt = euclidean_distance(eye_data_downsampled_task_all, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_1 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_1.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    

                
                elif run == 1: 
                    sub_run_X_pt = np.array(df_pred_median_pt['X'])[313:626]
                    sub_run_Y_pt = np.array(df_pred_median_pt['Y'])[313:626]
                    sub_run_Y_pt = -1 * sub_run_Y_pt
                    
                    if task == "open":
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool]
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_1_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_1_trials_bool] 
                        
                        ee_pt = euclidean_distance(eye_data_downsampled_task_1, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_2 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task == "no_stim": 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_2_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_2_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_2, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_2 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

                    elif task ==  "blink":
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_3_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_3_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_3, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_2 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    
                    elif task == "all":
                        eye_data_downsampled_task_all = eye_data_downsampled 

                        # Pretrained
                        ee_pt = euclidean_distance(eye_data_downsampled_task_all, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_2 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_2.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                        
                        
                elif run == 2: 
                    sub_run_X_pt = np.array(df_pred_median_pt['X'])[626:]
                    sub_run_Y_pt = np.array(df_pred_median_pt['Y'])[626:]
                    sub_run_Y_pt = -1 * sub_run_Y_pt
                    

                    if task == "open":
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool] 
                        #Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_1_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_1_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_1, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_3 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                        
                
                    elif task == "no_stim": 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_2_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_2_trials_bool] 

                        ee_pt = euclidean_distance(eye_data_downsampled_task_2, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_3 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                        

                    elif task ==  "blink":
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_3_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_3_trials_bool] 
                        
                        ee_pt = euclidean_distance(eye_data_downsampled_task_3, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_3 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')
                    
                    elif task == "all":
                        eye_data_downsampled_task_all = eye_data_downsampled 

                        # Pretrained
                        ee_pt = euclidean_distance(eye_data_downsampled_task_all, sub_run_X_pt, sub_run_Y_pt)
                        eucl_dist_df_3 = pd.DataFrame(ee_pt, columns=['ee'])
                        ee_file_path = f'/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_0{run+1}_ee_pretrained.tsv.gz'
                        eucl_dist_df_3.to_csv(ee_file_path, sep='\t', index=False, compression='gzip')

In [11]:
# Calculate mean and 75th perc Pretrained EE 

# Define tasks
tasks = ["open", "no_stim", "blink", "all"]

# Initialize dictionary to store results
subject_ee_means_pt = {}
subject_ee_perc_pt = {}

for subject in subjects:
    subject_ee_means_pt[subject] = {}
    subject_ee_perc_pt[subject] = {}
    
    for task in tasks:
        # Load data for each run
        ee_run_01 = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_01_ee_calibpretrained.tsv.gz", compression='gzip', delimiter='\t')[['ee']].to_numpy()
        ee_run_02 = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_02_ee_calibpretrained.tsv.gz", compression='gzip', delimiter='\t')[['ee']].to_numpy()
        ee_run_03 = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_03_ee_calibpretrained.tsv.gz", compression='gzip', delimiter='\t')[['ee']].to_numpy()
        
        # Compute mean across all runs
        all_ee = np.concatenate([ee_run_01, ee_run_02, ee_run_03])
        mean_ee = np.mean(all_ee)
        perc_ee = np.percentile(all_ee, 75)
        print(f"{task}, {subject}: {perc_ee}")
        
        # Save in dictionaries
        subject_ee_means_pt[subject][task] = mean_ee
        subject_ee_perc_pt[subject][task] = perc_ee



open, sub-03: 7.127056817669695
no_stim, sub-03: 7.455522443228745
blink, sub-03: 5.498622001352352
all, sub-03: 6.776752222497136
open, sub-04: 6.030446826315589
no_stim, sub-04: 4.750427146724457
blink, sub-04: 6.578452374878304
all, sub-04: 5.879772680823905
open, sub-05: 7.896719143973314
no_stim, sub-05: 7.600295758687048
blink, sub-05: 6.584088447088028
all, sub-05: 7.362063896985524
open, sub-06: 6.864988816212846
no_stim, sub-06: 7.433708912886621
blink, sub-06: 7.421244099403343
all, sub-06: 7.28492453348644
open, sub-07: 7.846121048176179
no_stim, sub-07: 8.296402210993287
blink, sub-07: 7.728854922384813
all, sub-07: 7.103114513467348
open, sub-09: 7.68279592706917
no_stim, sub-09: 6.991148623555404
blink, sub-09: 5.890750671765224
all, sub-09: 6.9735454377159405
open, sub-11: 15.561373467077473
no_stim, sub-11: 13.821678873584025
blink, sub-11: 6.301773276365054
all, sub-11: 8.480916718165332
open, sub-13: 6.5100709193853925
no_stim, sub-13: 7.115788016245314
blink, sub-13:

In [12]:
# Calculate mean and 75th perc of Original Pretrained

# Define tasks
tasks = ["open", "no_stim", "blink", "all"]

# Initialize dictionary to store results
subject_ee_means_pt_old = {}
subject_ee_perc_pt_old = {}

for subject in subjects:
    subject_ee_means_pt_old[subject] = {}
    subject_ee_perc_pt_old[subject] = {}
    
    for task in tasks:
        # Load data for each run
        ee_run_01 = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_01_ee_pretrained.tsv.gz", compression='gzip', delimiter='\t')[['ee']].to_numpy()
        ee_run_02 = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_02_ee_pretrained.tsv.gz", compression='gzip', delimiter='\t')[['ee']].to_numpy()
        ee_run_03 = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_subtask-{task}_run_03_ee_pretrained.tsv.gz", compression='gzip', delimiter='\t')[['ee']].to_numpy()
        
        # Compute mean across all runs
        all_ee = np.concatenate([ee_run_01, ee_run_02, ee_run_03])
        mean_ee = np.mean(all_ee)
        perc_ee = np.percentile(all_ee, 75)
        print(f"{task}, {subject}: {mean_ee}")
        
        # Save in dictionaries
        subject_ee_means_pt_old[subject][task] = mean_ee
        subject_ee_perc_pt_old[subject][task] = perc_ee


open, sub-03: 8.936110800572335
no_stim, sub-03: 8.378469073066576
blink, sub-03: 5.7620663663341825
all, sub-03: 6.572473814153117
open, sub-04: 6.301010709109105
no_stim, sub-04: 6.3228973524142
blink, sub-04: 3.9102556995417492
all, sub-04: 4.756836898339048
open, sub-05: 8.90358217466503
no_stim, sub-05: 8.904329257663882
blink, sub-05: 5.633266940922868
all, sub-05: 6.617779832363439
open, sub-06: 9.559368075983159
no_stim, sub-06: 9.824099793053497
blink, sub-06: 7.640455254035183
all, sub-06: 7.624146808588608
open, sub-07: 10.499141032014807
no_stim, sub-07: 9.74414227690109
blink, sub-07: 8.15638760222176
all, sub-07: 7.819635142379997
open, sub-09: 11.125146322706286
no_stim, sub-09: 10.433430841649194
blink, sub-09: 7.141944967911051
all, sub-09: 7.81037291939143
open, sub-11: 13.13422410787052
no_stim, sub-11: 13.134138212800309
blink, sub-11: 6.341746410864889
all, sub-11: 9.049638411768333
open, sub-13: 7.610430164958944
no_stim, sub-13: 7.3721681815775915
blink, sub-13: 

In [13]:
def compute_pearson_correlation(eye_data, model_data_x, model_data_y, slice_range, flip_y=False):
    """
    Computes the mean Pearson correlation for X and Y model predictions and eye tracking data.
    
    Arguments:
        eye_data: 2D NumPy array with eye tracking data (columns: timestamps, x, y).
        model_data_x: Model predictions for X coordinate.
        model_data_y: Model predictions for Y coordinate.
        slice_range: Tuple indicating (start, end) indices for slicing model predictions.
        flip_y: If True, flips the Y coordinate of model predictions.

    Returns:
        Mean Pearson correlation of X and Y predictions.
    """
    start, end = slice_range
    model_data_x = model_data_x[start:end]
    model_data_y = model_data_y[start:end]
    
    if flip_y:
        model_data_y = -1 * model_data_y  # Flip the Y coordinate if needed
    
    # Pearson correlation for X and Y
    pearson_x = np.corrcoef(eye_data[:, 0], model_data_x)[0, 1]
    pearson_y = np.corrcoef(eye_data[:, 1], model_data_y)[0, 1]
    
    return (pearson_x + pearson_y) / 2

In [14]:
# CORRELATIONS FOR TASK ALL
from scipy.stats import pearsonr
import numpy as np
import pandas as pd


runs = 3
slices = [(None,313), (313, 626), (626, None)]  # Corresponding slice ranges per run

correlation_ft, correlation_pt, correlation_sc = [], [], []
subject_correlations_ft = {sub: [] for sub in subjects}
subject_correlations_pt = {sub: [] for sub in subjects}
subject_correlations_pt_old = {sub: [] for sub in subjects}

# Load model evaluations
evaluation_ft = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed.npy", allow_pickle=True).item()
evaluation_pt = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed_calibpretrained.npy", allow_pickle=True).item()
evaluation_pt_old = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed_pretrained.npy", allow_pickle=True).item()

for subject in subjects:
    for run in range(runs):
        # Load and downsample eye tracking data
        eye_data = pd.read_csv(
            f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_run_0{run+1}_eyedata.tsv.gz",
            compression='gzip', delimiter='\t'
        )[['timestamps', 'x', 'y']].to_numpy()
        
        #eye_data_downsampled = downsample_to_targetrate(eye_data, 1000, 8.33)
        eye_data_downsampled_x = chunk_and_median(eye_data[:,1])
        
        eye_data_downsampled_y = chunk_and_median(eye_data[:,2])
        eye_data_downsampled = np.stack((eye_data_downsampled_x, eye_data_downsampled_y), axis=1)
        

        # Load model predictions
        subject_data_ft = evaluation_ft[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data/{subject}_DeepMReyeClosed_label.npz"]
        df_pred_median_ft, df_pred_subtr_ft = adapt_evaluation(subject_data_ft)

        subject_data_pt = evaluation_pt[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data_pretrained/{subject}_DeepMReyeClosed_no_label.npz"]
        df_pred_median_pt, df_pred_subtr_pt = adapt_evaluation(subject_data_pt)

        subject_data_pt_old = evaluation_pt_old[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data_pretrained/{subject}_DeepMReyeClosed_no_label.npz"]
        df_pred_median_pt_old, df_pred_subtr_pt_old = adapt_evaluation(subject_data_pt_old)

        

        # Slice predictions according to the current run
        slice_range = slices[run]

        # Fine-Tuned Model
        pearson_mean_ft = compute_pearson_correlation(
            eye_data_downsampled, np.array(df_pred_median_ft['X']), np.array(df_pred_median_ft['Y']),
            slice_range
        )
        correlation_ft.append(pearson_mean_ft)
        subject_correlations_ft[subject].append(pearson_mean_ft)

        # Calib Pretrained Model 
        pearson_mean_pt = compute_pearson_correlation(
            eye_data_downsampled, np.array(df_pred_median_pt['X']), np.array(df_pred_median_pt['Y']),
            slice_range, flip_y=False
        )
        correlation_pt.append(pearson_mean_pt)
        subject_correlations_pt[subject].append(pearson_mean_pt)

        # Original Pretrained Model (flipping Y)
        pearson_mean_pt_old = compute_pearson_correlation(
            eye_data_downsampled, np.array(df_pred_median_pt_old['X']), np.array(df_pred_median_pt_old['Y']),
            slice_range, flip_y=True
        )
        correlation_sc.append(pearson_mean_pt_old)
        subject_correlations_pt_old[subject].append(pearson_mean_pt_old)

        # Print individual correlations for this run
        print(f"Pearson correlation {subject}, Run {run+1}: FT={pearson_mean_ft:.4f}, , PT O={pearson_mean_pt_old:.4f}, PT Ca={pearson_mean_pt:.4f}") #

# Compute mean correlations per subject
mean_correlations_per_subject_ft = {sub: np.mean(corrs) for sub, corrs in subject_correlations_ft.items() if corrs}
mean_correlations_per_subject_pt = {sub: np.mean(corrs) for sub, corrs in subject_correlations_pt.items() if corrs}
mean_correlations_per_subject_pt_old = {sub: np.mean(corrs) for sub, corrs in subject_correlations_pt_old.items() if corrs}


Pearson correlation sub-03, Run 1: FT=0.8200, , PT O=0.6100, PT Ca=0.7299
Pearson correlation sub-03, Run 2: FT=0.8431, , PT O=0.6848, PT Ca=0.7626
Pearson correlation sub-03, Run 3: FT=0.8157, , PT O=0.6884, PT Ca=0.7623
Pearson correlation sub-04, Run 1: FT=0.7062, , PT O=0.6289, PT Ca=0.6329
Pearson correlation sub-04, Run 2: FT=0.7817, , PT O=0.6935, PT Ca=0.7080
Pearson correlation sub-04, Run 3: FT=0.7464, , PT O=0.6860, PT Ca=0.7030
Pearson correlation sub-05, Run 1: FT=0.7689, , PT O=0.6576, PT Ca=0.6681
Pearson correlation sub-05, Run 2: FT=0.7562, , PT O=0.6468, PT Ca=0.6828
Pearson correlation sub-05, Run 3: FT=0.7262, , PT O=0.6435, PT Ca=0.6393
Pearson correlation sub-06, Run 1: FT=0.8642, , PT O=0.6757, PT Ca=0.7520
Pearson correlation sub-06, Run 2: FT=0.8430, , PT O=0.6704, PT Ca=0.7467
Pearson correlation sub-06, Run 3: FT=0.8656, , PT O=0.7083, PT Ca=0.7771
Pearson correlation sub-07, Run 1: FT=0.7250, , PT O=0.5944, PT Ca=0.6763
Pearson correlation sub-07, Run 2: FT=

In [ ]:
# Correlations FOR TASK FIX, PUR, FV
from scipy.stats import pearsonr
import matplotlib.pyplot as plt
runs = 3

evaluation_ft = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed.npy", allow_pickle=True).item()
evaluation_pt = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed_calibpretrained.npy", allow_pickle=True).item()
evaluation_pt_old = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_closed/pred/evaluation_dict_closed_pretrained.npy", allow_pickle=True).item()

tasks = ["open", "blink", "no_stim"]

subject_task_correlations_ft = {subject: {task: [] for task in tasks} for subject in subjects}
subject_task_correlations_pt = {subject: {task: [] for task in tasks} for subject in subjects}
subject_task_correlations_pt_old = {subject: {task: [] for task in tasks} for subject in subjects}

for subject in subjects: 
    for run in range(runs): 
         # load eye tracking 
            eye_data = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/{subject}/eyetracking/{subject}_task-DeepMReyeClosed_run_0{run+1}_eyedata.tsv.gz", compression='gzip', delimiter='\t')
            eye_data = eye_data[['timestamps','x', 'y']].to_numpy()     
            # downsample eye data 
            #eye_data_downsampled = downsample_to_targetrate(eye_data, 1000, 8.33)
            #eye_data_downsampled = eye_data_downsampled[:1540,:]
            eye_data_downsampled_x = chunk_and_median(eye_data[:,1])
            eye_data_downsampled_y = chunk_and_median(eye_data[:,2])
            eye_data_downsampled = np.stack((eye_data_downsampled_x, eye_data_downsampled_y), axis=1)

            for task in tasks: 
                # Load model predictions
                subject_data_ft = evaluation_ft[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data/{subject}_DeepMReyeClosed_label.npz"]
                df_pred_median_ft, df_pred_subtr_ft = adapt_evaluation(subject_data_ft)

                subject_data_pt = evaluation_pt[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data_pretrained/{subject}_DeepMReyeClosed_no_label.npz"]
                df_pred_median_pt, df_pred_subtr_pt = adapt_evaluation(subject_data_pt)

                subject_data_pt_old = evaluation_pt_old[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_closed/pp_data_pretrained/{subject}_DeepMReyeClosed_no_label.npz"]
                df_pred_median_pt_old, df_pred_subtr_pt_old = adapt_evaluation(subject_data_pt_old)
                

                if run == 0: 
                    #Fine Tuned
                    sub_run_X_ft = np.array(df_pred_median_ft['X'][:313])
                    sub_run_Y_ft = np.array(df_pred_median_ft['Y'][:313])

                    # Pretrained
                    sub_run_X_pt = np.array(df_pred_median_pt['X'][:313])
                    sub_run_Y_pt = np.array(df_pred_median_pt['Y'][:313])
                    

                    # Pretrained
                    sub_run_X_pt_old = np.array(df_pred_median_pt['X'][:313])
                    sub_run_Y_pt_old = np.array(df_pred_median_pt['Y'][:313])
                    sub_run_Y_pt_old = -1 * sub_run_Y_pt_old

                    
                    
                    if task == 'open':
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_1_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_1_trials_bool] 
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool] 

                        pearson_x_ft = np.corrcoef(eye_data_downsampled_task_1[:, 0], sub_run_X_ft)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_1[:, 1], sub_run_Y_ft)[0, 1]
                        pearson_mean_ft = (pearson_x_ft + pearson_y_ft) / 2
                        subject_task_correlations_ft[subject][task].append(pearson_mean_ft)
                        
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_1_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_1_trials_bool] 
                        
                        pearson_x_pt = np.corrcoef(eye_data_downsampled_task_1[:, 0], sub_run_X_pt)[0, 1]
                        pearson_y_pt = np.corrcoef(eye_data_downsampled_task_1[:, 1], sub_run_Y_pt)[0, 1]
                        pearson_mean_pt = (pearson_x_pt + pearson_y_pt) / 2
                        subject_task_correlations_pt[subject][task].append(pearson_mean_pt)

                        # Pretrained old
                        sub_run_X_pt_old = sub_run_X_pt_old[task_1_trials_bool]   
                        sub_run_Y_pt_old = sub_run_Y_pt_old[task_1_trials_bool] 
                        
                        pearson_x_pt_old = np.corrcoef(eye_data_downsampled_task_1[:, 0], sub_run_X_pt_old)[0, 1]
                        pearson_y_pt_old = np.corrcoef(eye_data_downsampled_task_1[:, 1], sub_run_Y_pt_old)[0, 1]
                        pearson_mean_pt_old = (pearson_x_pt_old + pearson_y_pt_old) / 2
                        subject_task_correlations_pt_old[subject][task].append(pearson_mean_pt_old)

                        #print(f"Pearson correlation x {subject}: {pearson_x_sc:.4f}")
                        #print(f"Pearson correlation y {subject}: {pearson_y_sc:.4f}")

                    elif task == "blink": 
                        sub_run_X_ft = sub_run_X_ft[task_2_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_2_trials_bool] 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 

                        # Fine Tuned
                        pearson_x_ft = np.corrcoef(eye_data_downsampled_task_2[:, 0], sub_run_X_ft)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_2[:, 1], sub_run_Y_ft)[0, 1]
                        pearson_mean_ft = (pearson_x_ft + pearson_y_ft) / 2
                        subject_task_correlations_ft[subject][task].append(pearson_mean_ft)
                        
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_2_trials_bool]  
                        sub_run_Y_pt = sub_run_Y_pt[task_2_trials_bool]  
                        
                        pearson_x_pt = np.corrcoef(eye_data_downsampled_task_2[:, 0], sub_run_X_pt)[0, 1]
                        pearson_y_pt = np.corrcoef(eye_data_downsampled_task_2[:, 1], sub_run_Y_pt)[0, 1]
                        pearson_mean_pt = (pearson_x_pt + pearson_y_pt) / 2
                        subject_task_correlations_pt[subject][task].append(pearson_mean_pt)

                        # Pretrained old
                        sub_run_X_pt_old = sub_run_X_pt_old[task_2_trials_bool]   
                        sub_run_Y_pt_old = sub_run_Y_pt_old[task_2_trials_bool] 
                        
                        pearson_x_pt_old = np.corrcoef(eye_data_downsampled_task_2[:, 0], sub_run_X_pt_old)[0, 1]
                        pearson_y_pt_old = np.corrcoef(eye_data_downsampled_task_2[:, 1], sub_run_Y_pt_old)[0, 1]
                        pearson_mean_pt_old = (pearson_x_pt_old + pearson_y_pt_old) / 2
                        subject_task_correlations_pt_old[subject][task].append(pearson_mean_pt_old)

                        #print(f"Pearson correlation x {subject}: {pearson_x_sc:.4f}")
                        #print(f"Pearson correlation y {subject}: {pearson_y_sc:.4f}")

                    elif task == "no_stim":
                        sub_run_X_ft = sub_run_X_ft[task_3_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_3_trials_bool] 
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        # Fine Tuned
                        pearson_x_ft = np.corrcoef(eye_data_downsampled_task_3[:, 0], sub_run_X_ft)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_3[:, 1], sub_run_Y_ft)[0, 1]
                        pearson_mean_ft = (pearson_x_ft + pearson_y_ft) / 2
                        subject_task_correlations_ft[subject][task].append(pearson_mean_ft)
                        
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_3_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_3_trials_bool] 
                        

                        pearson_x_pt = np.corrcoef(eye_data_downsampled_task_3[:, 0], sub_run_X_pt)[0, 1]
                        pearson_y_pt = np.corrcoef(eye_data_downsampled_task_3[:, 1], sub_run_Y_pt)[0, 1]
                        pearson_mean_pt = (pearson_x_pt + pearson_y_pt) / 2
                        subject_task_correlations_pt[subject][task].append(pearson_mean_pt)


                        # Pretrained old
                        sub_run_X_pt_old = sub_run_X_pt_old[task_3_trials_bool]   
                        sub_run_Y_pt_old = sub_run_Y_pt_old[task_3_trials_bool] 
                        
                        pearson_x_pt_old = np.corrcoef(eye_data_downsampled_task_3[:, 0], sub_run_X_pt_old)[0, 1]
                        pearson_y_pt_old = np.corrcoef(eye_data_downsampled_task_3[:, 1], sub_run_Y_pt_old)[0, 1]
                        pearson_mean_pt_old = (pearson_x_pt_old + pearson_y_pt_old) / 2
                        subject_task_correlations_pt_old[subject][task].append(pearson_mean_pt_old)

                        #print(f"Pearson correlation x {subject}: {pearson_x_sc:.4f}")
                        #print(f"Pearson correlation y {subject}: {pearson_y_sc:.4f}")

                
                elif run == 1: 
                    #Fine Tuned
                    sub_run_X_ft = np.array(df_pred_median_ft['X'][313:626])
                    sub_run_Y_ft = np.array(df_pred_median_ft['Y'][313:626])

                    # Pretrained
                    sub_run_X_pt = np.array(df_pred_median_pt['X'][313:626])
                    sub_run_Y_pt = np.array(df_pred_median_pt['Y'][313:626])
                    sub_run_Y_pt = -1 * sub_run_Y_pt

                    # Pretrained
                    sub_run_X_pt_old = np.array(df_pred_median_pt['X'][313:626])
                    sub_run_Y_pt_old = np.array(df_pred_median_pt['Y'][313:626])
                    sub_run_Y_pt_old = -1 * sub_run_Y_pt_old
                   
                    
                    if task == 'open':
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_1_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_1_trials_bool] 

                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool] 

                        pearson_x_ft = np.corrcoef(eye_data_downsampled_task_1[:, 0], sub_run_X_ft)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_1[:, 1], sub_run_Y_ft)[0, 1]
                        pearson_mean_ft = (pearson_x_ft + pearson_y_ft) / 2
                        subject_task_correlations_ft[subject][task].append(pearson_mean_ft)
                        
                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_1_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_1_trials_bool] 
                        

                        pearson_x_pt = np.corrcoef(eye_data_downsampled_task_1[:, 0], sub_run_X_pt)[0, 1]
                        pearson_y_pt = np.corrcoef(eye_data_downsampled_task_1[:, 1], sub_run_Y_pt)[0, 1]
                        pearson_mean_pt = (pearson_x_pt + pearson_y_pt) / 2
                        subject_task_correlations_pt[subject][task].append(pearson_mean_pt)

                        # Pretrained old
                        sub_run_X_pt_old = sub_run_X_pt_old[task_1_trials_bool]   
                        sub_run_Y_pt_old = sub_run_Y_pt_old[task_1_trials_bool] 
                        
                        pearson_x_pt_old = np.corrcoef(eye_data_downsampled_task_1[:, 0], sub_run_X_pt_old)[0, 1]
                        pearson_y_pt_old = np.corrcoef(eye_data_downsampled_task_1[:, 1], sub_run_Y_pt_old)[0, 1]
                        pearson_mean_pt_old = (pearson_x_pt_old + pearson_y_pt_old) / 2
                        subject_task_correlations_pt_old[subject][task].append(pearson_mean_pt_old)

                        #print(f"Pearson correlation x {subject}: {pearson_x_sc:.4f}")
                        #print(f"Pearson correlation y {subject}: {pearson_y_sc:.4f}")

                    elif task == "blink": 
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_2_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_2_trials_bool] 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 

                        pearson_x_ft = np.corrcoef(eye_data_downsampled_task_2[:, 0], sub_run_X_ft)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_2[:, 1], sub_run_Y_ft)[0, 1]
                        pearson_mean_ft = (pearson_x_ft + pearson_y_ft) / 2
                        subject_task_correlations_ft[subject][task].append(pearson_mean_ft)

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_2_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_2_trials_bool] 

                        pearson_x_pt = np.corrcoef(eye_data_downsampled_task_2[:, 0], sub_run_X_pt)[0, 1]
                        pearson_y_pt = np.corrcoef(eye_data_downsampled_task_2[:, 1], sub_run_Y_pt)[0, 1]
                        pearson_mean_pt = (pearson_x_pt + pearson_y_pt) / 2
                        subject_task_correlations_pt[subject][task].append(pearson_mean_pt)

                        # Pretrained old
                        sub_run_X_pt_old = sub_run_X_pt_old[task_2_trials_bool]   
                        sub_run_Y_pt_old = sub_run_Y_pt_old[task_2_trials_bool] 
                        
                        pearson_x_pt_old = np.corrcoef(eye_data_downsampled_task_2[:, 0], sub_run_X_pt_old)[0, 1]
                        pearson_y_pt_old = np.corrcoef(eye_data_downsampled_task_2[:, 1], sub_run_Y_pt_old)[0, 1]
                        pearson_mean_pt_old = (pearson_x_pt_old + pearson_y_pt_old) / 2
                        subject_task_correlations_pt_old[subject][task].append(pearson_mean_pt_old)

                        #print(f"Pearson correlation x {subject}: {pearson_x_sc:.4f}")
                        #print(f"Pearson correlation y {subject}: {pearson_y_sc:.4f}")
                    elif task == "no_stim":
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_3_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_3_trials_bool] 
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        pearson_x_ft = np.corrcoef(eye_data_downsampled_task_3[:, 0], sub_run_X_ft)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_3[:, 1], sub_run_Y_ft)[0, 1]
                        pearson_mean_ft = (pearson_x_ft + pearson_y_ft) / 2
                        subject_task_correlations_ft[subject][task].append(pearson_mean_ft)

                        sub_run_X_pt = sub_run_X_pt[task_3_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_3_trials_bool] 

                       
                        # Pretrained
                        pearson_x_pt = np.corrcoef(eye_data_downsampled_task_3[:, 0], sub_run_X_pt)[0, 1]
                        pearson_y_pt = np.corrcoef(eye_data_downsampled_task_3[:, 1], sub_run_Y_pt)[0, 1]
                        pearson_mean_pt = (pearson_x_pt + pearson_y_pt) / 2
                        subject_task_correlations_pt[subject][task].append(pearson_mean_pt)


                        # Pretrained old
                        sub_run_X_pt_old = sub_run_X_pt_old[task_3_trials_bool]   
                        sub_run_Y_pt_old = sub_run_Y_pt_old[task_3_trials_bool] 
                        
                        pearson_x_pt_old = np.corrcoef(eye_data_downsampled_task_3[:, 0], sub_run_X_pt_old)[0, 1]
                        pearson_y_pt_old = np.corrcoef(eye_data_downsampled_task_3[:, 1], sub_run_Y_pt_old)[0, 1]
                        pearson_mean_pt_old = (pearson_x_pt_old + pearson_y_pt_old) / 2
                        subject_task_correlations_pt_old[subject][task].append(pearson_mean_pt_old)

                        #print(f"Pearson correlation x {subject}: {pearson_x_sc:.4f}")
                        #print(f"Pearson correlation y {subject}: {pearson_y_sc:.4f}")

                        
                elif run == 2: 
                    #Fine Tuned
                    sub_run_X_ft = np.array(df_pred_median_ft['X'][626:])
                    sub_run_Y_ft = np.array(df_pred_median_ft['Y'][626:])
                    
                    # Pretrained
                    sub_run_X_pt = np.array(df_pred_median_pt['X'][626:])
                    sub_run_Y_pt = np.array(df_pred_median_pt['Y'][626:])
                    sub_run_Y_pt = -1 * sub_run_Y_pt

                    # Pretrained
                    sub_run_X_pt_old = np.array(df_pred_median_pt['X'][626:])
                    sub_run_Y_pt_old = np.array(df_pred_median_pt['Y'][626:])
                    sub_run_Y_pt_old = -1 * sub_run_Y_pt_old
                   

                    if task == 'open':
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_1_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_1_trials_bool] 
                        eye_data_downsampled_task_1 = eye_data_downsampled[task_1_trials_bool] 

                        pearson_x_ft = np.corrcoef(eye_data_downsampled_task_1[:, 0], sub_run_X_ft)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_1[:, 1], sub_run_Y_ft)[0, 1]
                        pearson_mean_ft = (pearson_x_ft + pearson_y_ft) / 2
                        subject_task_correlations_ft[subject][task].append(pearson_mean_ft)

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_1_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_1_trials_bool] 
                        

                        pearson_x_pt = np.corrcoef(eye_data_downsampled_task_1[:, 0], sub_run_X_pt)[0, 1]
                        pearson_y_pt = np.corrcoef(eye_data_downsampled_task_1[:, 1], sub_run_Y_pt)[0, 1]
                        pearson_mean_pt = (pearson_x_pt + pearson_y_pt) / 2
                        subject_task_correlations_pt[subject][task].append(pearson_mean_pt)

                        # Pretrained old
                        sub_run_X_pt_old = sub_run_X_pt_old[task_1_trials_bool]   
                        sub_run_Y_pt_old = sub_run_Y_pt_old[task_1_trials_bool] 
                        
                        pearson_x_pt_old = np.corrcoef(eye_data_downsampled_task_1[:, 0], sub_run_X_pt_old)[0, 1]
                        pearson_y_pt_old = np.corrcoef(eye_data_downsampled_task_1[:, 1], sub_run_Y_pt_old)[0, 1]
                        pearson_mean_pt_old = (pearson_x_pt_old + pearson_y_pt_old) / 2
                        subject_task_correlations_pt_old[subject][task].append(pearson_mean_pt_old)

                        #print(f"Pearson correlation x {subject}: {pearson_x_sc:.4f}")
                        #print(f"Pearson correlation y {subject}: {pearson_y_sc:.4f}")
                
                    elif task == "blink": 
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_2_trials_bool]   
                        sub_run_Y_ft = sub_run_Y_ft[task_2_trials_bool] 
                        eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 

                        pearson_x_ft = np.corrcoef(eye_data_downsampled_task_2[:, 0], sub_run_X_ft)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_2[:, 1], sub_run_Y_ft)[0, 1]
                        pearson_mean_ft = (pearson_x_ft + pearson_y_ft) / 2
                        subject_task_correlations_ft[subject][task].append(pearson_mean_ft)

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_2_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_2_trials_bool] 
                        

                        pearson_x_pt = np.corrcoef(eye_data_downsampled_task_2[:, 0], sub_run_X_pt)[0, 1]
                        pearson_y_pt = np.corrcoef(eye_data_downsampled_task_2[:, 1], sub_run_Y_pt)[0, 1]
                        pearson_mean_pt = (pearson_x_pt + pearson_y_pt) / 2
                        subject_task_correlations_pt[subject][task].append(pearson_mean_pt)


                        # Pretrained old
                        sub_run_X_pt_old = sub_run_X_pt_old[task_2_trials_bool]   
                        sub_run_Y_pt_old = sub_run_Y_pt_old[task_2_trials_bool] 
                        
                        pearson_x_pt_old = np.corrcoef(eye_data_downsampled_task_2[:, 0], sub_run_X_pt_old)[0, 1]
                        pearson_y_pt_old = np.corrcoef(eye_data_downsampled_task_2[:, 1], sub_run_Y_pt_old)[0, 1]
                        pearson_mean_pt_old = (pearson_x_pt_old + pearson_y_pt_old) / 2
                        subject_task_correlations_pt_old[subject][task].append(pearson_mean_pt_old)

                        #print(f"Pearson correlation x {subject}: {pearson_x_sc:.4f}")
                        #print(f"Pearson correlation y {subject}: {pearson_y_sc:.4f}")

                    elif task == "no_stim":
                        # Fine Tuned
                        sub_run_X_ft = sub_run_X_ft[task_3_trials_bool]  
                        sub_run_Y_ft = sub_run_Y_ft[task_3_trials_bool]  
                        eye_data_downsampled_task_3 = eye_data_downsampled[task_3_trials_bool] 

                        pearson_x_ft = np.corrcoef(eye_data_downsampled_task_3[:, 0], sub_run_X_ft)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_3[:, 1], sub_run_Y_ft)[0, 1]
                        pearson_mean_ft = (pearson_x_ft + pearson_y_ft) / 2
                        subject_task_correlations_ft[subject][task].append(pearson_mean_ft)

                        # Pretrained
                        sub_run_X_pt = sub_run_X_pt[task_3_trials_bool]   
                        sub_run_Y_pt = sub_run_Y_pt[task_3_trials_bool] 
                        

                        pearson_x_pt = np.corrcoef(eye_data_downsampled_task_3[:, 0], sub_run_X_pt)[0, 1]
                        pearson_y_ft = np.corrcoef(eye_data_downsampled_task_3[:, 1], sub_run_Y_pt)[0, 1]
                        pearson_mean_pt = (pearson_x_pt + pearson_y_pt) / 2
                        subject_task_correlations_pt[subject][task].append(pearson_mean_pt)


                        # Pretrained old
                        sub_run_X_pt_old = sub_run_X_pt_old[task_3_trials_bool]   
                        sub_run_Y_pt_old = sub_run_Y_pt_old[task_3_trials_bool] 
                        
                        pearson_x_pt_old = np.corrcoef(eye_data_downsampled_task_3[:, 0], sub_run_X_pt_old)[0, 1]
                        pearson_y_pt_old = np.corrcoef(eye_data_downsampled_task_3[:, 1], sub_run_Y_pt_old)[0, 1]
                        pearson_mean_pt_old = (pearson_x_pt_old + pearson_y_pt_old) / 2
                        subject_task_correlations_pt_old[subject][task].append(pearson_mean_pt_old)

                        #print(f"Pearson correlation x {subject}: {pearson_x_sc:.4f}")
                        #print(f"Pearson correlation y {subject}: {pearson_y_sc:.4f}")
                    

mean_correlations_per_subject_task_ft = {
    sub: {#
        task: np.mean(corrs) if corrs else None  
        for task, corrs in task_dict.items()
    }
    for sub, task_dict in subject_task_correlations_ft.items()
}     

mean_correlations_per_subject_task_pt = {
    sub: {
        task: np.mean(corrs) if corrs else None  
        for task, corrs in task_dict.items()
    }
    for sub, task_dict in subject_task_correlations_pt.items()
}  

mean_correlations_per_subject_task_pt_old = {
    sub: {
        task: np.mean(corrs) if corrs else None  
        for task, corrs in task_dict.items()
    }
    for sub, task_dict in subject_task_correlations_pt_old.items()
}  
    

NameError: name 'pearson_x_sc' is not defined

In [17]:
# Adding data under the task "all"
for subject, value in mean_correlations_per_subject_ft.items():
    if subject not in mean_correlations_per_subject_task_ft:
        mean_correlations_per_subject_task_ft[subject] = {}
    mean_correlations_per_subject_task_ft[subject]['all'] = value

# Now task_data includes the "all" data for each subject
print(mean_correlations_per_subject_task_ft)

{'sub-02': {'fix': 0.8887784643117165, 'pur': 0.8229763633059468, 'fv': 0.5727323937993797, 'all': 0.846730734012019}, 'sub-03': {'fix': 0.8684284592946067, 'pur': 0.8604579355007984, 'fv': 0.718759043831604, 'all': 0.8494515955454162}, 'sub-04': {'fix': 0.9247870789006009, 'pur': 0.8722671816903537, 'fv': 0.5897538482869814, 'all': 0.8843833024581892}, 'sub-05': {'fix': 0.8361743029764054, 'pur': 0.8307431525647484, 'fv': 0.5578243733976768, 'all': 0.8007594784793186}, 'sub-06': {'fix': 0.9485643036974176, 'pur': 0.9022592394985555, 'fv': 0.8089441155700033, 'all': 0.9194114559771362}, 'sub-07': {'fix': 0.7369002311182409, 'pur': 0.8503850725946825, 'fv': 0.5792895769468757, 'all': 0.7593889013539373}, 'sub-08': {'fix': 0.7878267023161983, 'pur': 0.8004766617567786, 'fv': 0.6144493105342248, 'all': 0.7909838616662884}, 'sub-09': {'fix': 0.8904969882011299, 'pur': 0.9061790364663155, 'fv': 0.7685281585009346, 'all': 0.8861212424414515}, 'sub-11': {'fix': 0.8979883209836635, 'pur': 0.82

In [18]:
# Adding data under the task "all"
for subject, value in mean_correlations_per_subject_pt.items():
    if subject not in mean_correlations_per_subject_task_pt:
        mean_correlations_per_subject_task_pt[subject] = {}
    mean_correlations_per_subject_task_pt[subject]['all'] = value

# Now task_data includes the "all" data for each subject
print(mean_correlations_per_subject_task_pt)

{'sub-02': {'fix': 0.8437982551119232, 'pur': 0.7884716734238335, 'fv': 0.532823700205452, 'all': 0.771044743870284}, 'sub-03': {'fix': 0.7762016604551706, 'pur': 0.8455658404065128, 'fv': 0.7525199730979145, 'all': 0.7815308638599184}, 'sub-04': {'fix': 0.9351317670839098, 'pur': 0.8892381179125408, 'fv': 0.688584132659761, 'all': 0.8931581414410458}, 'sub-05': {'fix': 0.8214247390461397, 'pur': 0.8185105213894098, 'fv': 0.6785154326675015, 'all': 0.8048170523236514}, 'sub-06': {'fix': 0.9435561268442397, 'pur': 0.8683521007640346, 'fv': 0.7882280557121025, 'all': 0.8960317376295838}, 'sub-07': {'fix': 0.6852606550853396, 'pur': 0.8161185690177243, 'fv': 0.5931275174164338, 'all': 0.702954310742823}, 'sub-08': {'fix': 0.7041824338015958, 'pur': 0.777631396450455, 'fv': 0.6436394239934878, 'all': 0.7284074660005411}, 'sub-09': {'fix': 0.8675722470239071, 'pur': 0.8886795118648868, 'fv': 0.7762887440081379, 'all': 0.8679455407074341}, 'sub-11': {'fix': 0.8977880197195084, 'pur': 0.83552

In [19]:
# Adding data under the task "all"
for subject, value in mean_correlations_per_subject_sc.items():
    if subject not in mean_correlations_per_subject_task_sc:
        mean_correlations_per_subject_task_sc[subject] = {}
    mean_correlations_per_subject_task_sc[subject]['all'] = value

# Now task_data includes the "all" data for each subject
print(mean_correlations_per_subject_task_sc)

{'sub-02': {'fix': 0.843798255412139, 'pur': 0.7884716716574692, 'fv': 0.4274897644733418, 'all': 0.7710447425381153}, 'sub-03': {'fix': 0.7762016599618651, 'pur': 0.8455658390606051, 'fv': 0.6618229250113145, 'all': 0.781530863508728}, 'sub-04': {'fix': 0.9351317668858709, 'pur': 0.8892381177408225, 'fv': 0.6593633083397705, 'all': 0.8931581412827004}, 'sub-05': {'fix': 0.8214247397700071, 'pur': 0.8185105198599318, 'fv': 0.534222118297701, 'all': 0.8048170520988798}, 'sub-06': {'fix': 0.9435561265202669, 'pur': 0.8683521013515868, 'fv': 0.7755717876114385, 'all': 0.8960317377838879}, 'sub-07': {'fix': 0.6852606549889875, 'pur': 0.8161185692387162, 'fv': 0.560244018096395, 'all': 0.7029543106661765}, 'sub-08': {'fix': 0.7041824354419574, 'pur': 0.77763139784172, 'fv': 0.6360751997463902, 'all': 0.7284074672865355}, 'sub-09': {'fix': 0.8675722476792872, 'pur': 0.8886795124509069, 'fv': 0.7857120777828749, 'all': 0.8679455414485583}, 'sub-11': {'fix': 0.8977880194822209, 'pur': 0.835526

In [20]:
# Flatten dictionaries for each task
def flatten_dicts(model_results, task):
    correlations = []
    for subject in model_results:
        if task in model_results[subject]:
            correlations.append(model_results[subject][task])
    return np.array(correlations)

def statistic(x, y, axis):
    return np.mean(x, axis=axis) - np.mean(y, axis=axis)

In [21]:
# PERMUTATION TEST CORRELATION FT VS PT
from scipy.stats import permutation_test

tasks = ['fix', 'pur', 'fv', 'all']

for task in tasks:
    data1 = flatten_dicts(mean_correlations_per_subject_task_ft, task)
    data2 = flatten_dicts(mean_correlations_per_subject_task_pt, task)
    print(data1)

    res = permutation_test((data1, data2), statistic, permutation_type = 'samples',vectorized = True, n_resamples=10000, alternative='two-sided')
    print("Correlation FT vs. PT")
    print(f"Task: {task}")
    print(f"statistic: {res.statistic}")
    print(f"P-value: {res.pvalue:.4f}")
    print("-" * 30)


[0.88877846 0.86842846 0.92478708 0.8361743  0.9485643  0.73690023
 0.7878267  0.89049699 0.89798832 0.83135825 0.82604639 0.90649265]
Correlation FT vs. PT
Task: fix
statistic: 0.03820371105194753
P-value: 0.0024
------------------------------
[0.82297636 0.86045794 0.87226718 0.83074315 0.90225924 0.85038507
 0.80047666 0.90617904 0.82347317 0.80345771 0.87402784 0.89724925]
Correlation FT vs. PT
Task: pur
statistic: 0.016973183740722098
P-value: 0.0220
------------------------------
[0.57273239 0.71875904 0.58975385 0.55782437 0.80894412 0.57928958
 0.61444931 0.76852816 0.72705969 0.33380795 0.68144491 0.75423503]
Correlation FT vs. PT
Task: fv
statistic: -0.015823050652582937
P-value: 0.3472
------------------------------
[0.84673073 0.8494516  0.8843833  0.80075948 0.91941146 0.7593889
 0.79098386 0.88612124 0.85649814 0.8004429  0.81408592 0.89620289]
Correlation FT vs. PT
Task: all
statistic: 0.03489031319164004
P-value: 0.0044
------------------------------


In [22]:
# PERMUTATION TEST EE FT vs PT
tasks = ['fixation', 'pursuit', 'freeview', 'all']
for task in tasks:
    data1 = flatten_dicts(subject_ee_means_ft, task)
    data2 = flatten_dicts(subject_ee_means_pt, task)
    

    res = permutation_test((data1, data2), statistic, permutation_type = 'samples',vectorized = True, n_resamples=10000, alternative='two-sided')
    print("EE FT vs. PT")
    print(f"Task: {task}")
    print(f"statistic: {res.statistic}")
    print(f"P-value: {res.pvalue:.4f}")
    print("-" * 30)


EE FT vs. PT
Task: fixation
statistic: -0.8269206499524846
P-value: 0.0005
------------------------------
EE FT vs. PT
Task: pursuit
statistic: -0.26034409499188627
P-value: 0.0029
------------------------------
EE FT vs. PT
Task: freeview
statistic: -0.02530252139741407
P-value: 0.7920
------------------------------
EE FT vs. PT
Task: all
statistic: -0.47715984603393524
P-value: 0.0005
------------------------------


In [23]:
# PERMUTATION TEST EE FT vs SC
from scipy.stats import permutation_test

tasks = ['fixation', 'pursuit', 'freeview', "all"]
for task in tasks:
    data1 = flatten_dicts(subject_ee_means_ft, task)
    data2 = flatten_dicts(subject_ee_means_sc, task)
    

    res = permutation_test((data1, data2), statistic, permutation_type = 'samples',vectorized = True, n_resamples=10000, alternative='two-sided')
    print("EE FT vs. SC")
    print(f"Task: {task}")
    print(f"statistic: {res.statistic}")
    print(f"P-value: {res.pvalue:.4f}")
    print("-" * 30)


EE FT vs. SC
Task: fixation
statistic: -0.39440187495254664
P-value: 0.0059
------------------------------
EE FT vs. SC
Task: pursuit
statistic: -0.130111900264271
P-value: 0.0752
------------------------------
EE FT vs. SC
Task: freeview
statistic: -0.20278352144509038
P-value: 0.0586
------------------------------
EE FT vs. SC
Task: all
statistic: -0.2502590854621123
P-value: 0.0059
------------------------------


In [24]:
# PERMUTATION TEST EE PT vs SC
tasks = ['fixation', 'pursuit', 'freeview', 'all']
for task in tasks:
    data1 = flatten_dicts(subject_ee_means_pt, task)
    data2 = flatten_dicts(subject_ee_means_sc, task)
    

    res = permutation_test((data1, data2), statistic, permutation_type = 'samples',vectorized = True, n_resamples=10000, alternative='two-sided')
    print("EE PT vs. SC")
    print(f"Task: {task}")
    print(f"statistic: {res.statistic}")
    print(f"P-value: {res.pvalue:.4f}")
    print("-" * 30)


EE PT vs. SC
Task: fixation
statistic: 0.43251877499993796
P-value: 0.0010
------------------------------
EE PT vs. SC
Task: pursuit
statistic: 0.13023219472761527
P-value: 0.0972
------------------------------
EE PT vs. SC
Task: freeview
statistic: -0.1774810000476763
P-value: 0.0488
------------------------------
EE PT vs. SC
Task: all
statistic: 0.22690076057182296
P-value: 0.0034
------------------------------


In [ ]:
import plotly.graph_objects as go
import pandas as pd
from plotly.subplots import make_subplots


# Prepare a list of dictionaries for the DataFrame
tasks = ['fix', 'pur', 'fv', 'all']


data_list = []
for subject in subjects:
    for task in tasks:
        data_list.append({
            'subject': subject,
            'task': task,
            'finetuned': mean_correlations_per_subject_task_ft[subject].get(task, None),
            'pretrained': mean_correlations_per_subject_task_pt[subject].get(task, None),
            'scaled': mean_correlations_per_subject_task_sc[subject].get(task, None)
        })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data_list)

# Create subplots with 2 rows and 2 columns
fig = make_subplots(
    rows=2, cols=2,  # Two rows, two columns
    subplot_titles=["Fixation task", "Pursuit task", "Freeview task", "All tasks"],
    shared_yaxes=True
)

# Add the boxplots for each task and model
fig.add_trace(go.Box(
    y=df[df['task'] == 'fix']['finetuned'],
    name='Fine-tuned',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#069D6B'),  # Fine-tuned color
    width=0.4
), row=1, col=1)

fig.add_trace(go.Box(
    y=df[df['task'] == 'fix']['pretrained'],
    name='Pretrained',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#5AC7DB'),  # Pretrained color
    width=0.4
), row=1, col=1)

fig.add_trace(go.Box(
    y=df[df['task'] == 'fix']['scaled'],
    name='Scaled',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#BCA3AC'),  # Scaled color
    width=0.4
), row=1, col=1)

fig.add_trace(go.Box(
    y=df[df['task'] == 'pur']['finetuned'],
    name='Fine-tuned',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#069D6B'),  # Fine-tuned color
    width=0.4
), row=1, col=2)

fig.add_trace(go.Box(
    y=df[df['task'] == 'pur']['pretrained'],
    name='Pretrained',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#5AC7DB'),  # Pretrained color
    width=0.4
), row=1, col=2)

fig.add_trace(go.Box(
    y=df[df['task'] == 'pur']['scaled'],
    name='Scaled',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#BCA3AC'),  # Scaled color
    width=0.4
), row=1, col=2)

fig.add_trace(go.Box(
    y=df[df['task'] == 'fv']['finetuned'],
    name='Fine-tuned',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#069D6B'),  # Fine-tuned color
    width=0.4
), row=2, col=1)

fig.add_trace(go.Box(
    y=df[df['task'] == 'fv']['pretrained'],
    name='Pretrained',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#5AC7DB'),  # Pretrained color
    width=0.4
), row=2, col=1)

fig.add_trace(go.Box(
    y=df[df['task'] == 'fv']['scaled'],
    name='Scaled',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#BCA3AC'),  # Scaled color
    width=0.4
), row=2, col=1)

fig.add_trace(go.Box(
    y=df[df['task'] == 'all']['finetuned'],
    name='Fine-tuned',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#069D6B'),  # Fine-tuned color
    width=0.4
), row=2, col=2)

fig.add_trace(go.Box(
    y=df[df['task'] == 'all']['pretrained'],
    name='Pretrained',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#5AC7DB'),  # Pretrained color
    width=0.4
), row=2, col=2)

fig.add_trace(go.Box(
    y=df[df['task'] == 'all']['scaled'],
    name='Scaled',
    boxpoints="all",
    boxmean=False,
    marker=dict(color='#BCA3AC'),  # Scaled color
    width=0.4
), row=2, col=2)

# Update layout to improve spacing and add axis labels
fig.update_layout(
    title_text="Model Comparison for Different Tasks",
    height=1000,  # Set the height of the plot
    width=1500,  # Set the width of the plot
    showlegend=False,
    template="simple_white",  # Apply the simple white template
    boxmode="group",  # Group the boxes by model within each task
    yaxis_title="Correlation"
)

fig.show()
#fig.write_image("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_calib/figures/group/correlation_calib_boxplots.pdf")


In [25]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

# Prepare a list of dictionaries for the DataFrame
tasks = ['fix', 'pur', 'fv', 'all']

data_list = []
for subject in subjects:
    for task in tasks:
        data_list.append({
            'subject': subject,
            'task': task,
            'finetuned': mean_correlations_per_subject_task_ft[subject].get(task, None),
            'pretrained': mean_correlations_per_subject_task_pt[subject].get(task, None),
            'scaled': mean_correlations_per_subject_task_sc[subject].get(task, None)
            
        })
        #

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data_list)

# Reshape data into long format for easier plotting
df_long = df.melt(id_vars=['subject', 'task'], value_vars=['pretrained','scaled','finetuned'], 
                   var_name='model', value_name='value')

# Define colors for each subject
colormap_subject_dict = {
    'sub-01': '#AA0DFE', 'sub-02': '#3283FE', 'sub-03': '#85660D', 'sub-04': '#782AB6',
    'sub-05': '#565656', 'sub-06': '#1C8356', 'sub-07': '#16FF32', 'sub-08': '#F7E1A0',
    'sub-09': '#E2E2E2', 'sub-11': '#1CBE4F', 'sub-13': '#DEA0FD', 'sub-14': '#FBE426', 'sub-15': '#325A9B'
}

# Create subplots with 2 rows and 2 columns
fig = make_subplots(
    rows=2, cols=2,  # Two rows, two columns
    subplot_titles=["Fixation task", "Pursuit task", "Freeview task", "All tasks"],
    shared_yaxes=False, 
    shared_xaxes=False
)

# Add strip plots, subject connections, and median/confidence intervals for each task
for i, task in enumerate(tasks):
    row, col = divmod(i, 2)
    row += 1  # Adjusting for subplot indexing
    col += 1
    
    task_df = df_long[df_long['task'] == task]
    
    # Create a strip plot for the data distribution
    strip_fig = px.strip(task_df, x='model', y='value', color_discrete_sequence = ['gray'], stripmode='overlay')
    
    for trace in strip_fig.data:
        fig.add_trace(trace, row=row, col=col)
    
    for subject in task_df['subject'].unique():
        subject_data = task_df[task_df['subject'] == subject].set_index('model').loc[['pretrained', 'scaled','finetuned']].reset_index()
        color = colormap_subject_dict.get(subject, 'gray')
        
        # Add subject connecting lines
        fig.add_trace(go.Scatter(
            x=subject_data['model'],
            y=subject_data['value'],
            mode='lines',
            opacity=0.7,
            line=dict(color=color, width=0.8),  
            name=subject,
            legendgroup=subject,
            showlegend=False
        ), row=row, col=col)
    
    # Compute median and 95% confidence interval
    for model in ['pretrained','scaled', 'finetuned']:
        model_data = task_df[task_df['model'] == model]['value']
        median = np.median(model_data)
        ci_lower, ci_upper = np.percentile(model_data, [2.5, 97.5])  # 95% CI
        
        # Add median marker
        fig.add_trace(go.Scatter(
            x=[model],
            y=[median],
            mode='markers',
            marker=dict(color='black', size=15, symbol='circle'),
            showlegend=False
        ), row=row, col=col)
        
        # Add confidence interval
        fig.add_trace(go.Scatter(
            x=[model, model],
            y=[ci_lower, ci_upper],
            mode='lines',
            line=dict(color='black', width=2),
            showlegend=False
        ), row=row, col=col)

# Update layout
fig.update_layout(
    height=1000,  # Set the height of the plot
    width=1500,  # Set the width of the plot
    showlegend=True,
    template="simple_white",
    yaxis_title="Correlation"
)

# Ensure y-axis range is consistent for all subplots
fig.update_yaxes(range=[0.1, 1], row=1, col=1)
fig.update_yaxes(range=[0.1, 1], row=1, col=2)
fig.update_yaxes(range=[0.1, 1], row=2, col=1)
fig.update_yaxes(range=[0.1, 1], row=2, col=2)

fig.show()
#fig.write_image("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_calib/figures/group/correlation_calib_plots.pdf")


In [26]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

# Prepare a list of dictionaries for the DataFrame
tasks = ['fixation', 'pursuit', 'freeview', 'all']

data_list = []
for subject in subjects:
    for task in tasks:
        data_list.append({
            'subject': subject,
            'task': task,
            'finetuned': subject_ee_means_ft[subject].get(task, None),
            'pretrained': subject_ee_means_pt[subject].get(task, None), 
            'scaled': subject_ee_means_sc[subject].get(task,None)
        })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data_list)

# Reshape data into long format for easier plotting
df_long = df.melt(id_vars=['subject', 'task'], value_vars=['pretrained', 'scaled', 'finetuned'], 
                   var_name='model', value_name='value')

# Define colors for each subject
colormap_subject_dict = {
    'sub-01': '#AA0DFE', 'sub-02': '#3283FE', 'sub-03': '#85660D', 'sub-04': '#782AB6',
    'sub-05': '#565656', 'sub-06': '#1C8356', 'sub-07': '#16FF32', 'sub-08': '#F7E1A0',
    'sub-09': '#E2E2E2', 'sub-11': '#1CBE4F', 'sub-13': '#DEA0FD', 'sub-14': '#FBE426', 'sub-15': '#325A9B'
}

# Create subplots with 2 rows and 2 columns
fig = make_subplots(
    rows=2, cols=2,  # Two rows, two columns
    subplot_titles=["Guided Fixation task", "Pursuit task", "Freeview task", "All tasks"],
    shared_yaxes=False, 
    shared_xaxes=False
)

# Add strip plots, subject connections, and median/confidence intervals for each task
for i, task in enumerate(tasks):
    row, col = divmod(i, 2)
    row += 1  # Adjusting for subplot indexing
    col += 1
    
    task_df = df_long[df_long['task'] == task]
    
    # Create a strip plot for the data distribution
    strip_fig = px.strip(task_df, x='model', y='value', color_discrete_sequence = ['gray'], stripmode='overlay')
    
    for trace in strip_fig.data:
        fig.add_trace(trace, row=row, col=col)
    
    for subject in task_df['subject'].unique():
        subject_data = task_df[task_df['subject'] == subject].set_index('model').loc[['pretrained', 'scaled', 'finetuned']].reset_index()
        color = colormap_subject_dict.get(subject, 'gray')
        
        # Add subject connecting lines
        fig.add_trace(go.Scatter(
            x=subject_data['model'],
            y=subject_data['value'],
            mode='lines',
            opacity=0.7,
            line=dict(color=color, width=0.8),  
            name=subject,
            legendgroup=subject,
            showlegend=False
        ), row=row, col=col)
    
    # Compute median and 95% confidence interval
    for model in ['pretrained', 'scaled', 'finetuned']:
        model_data = task_df[task_df['model'] == model]['value']
        median = np.median(model_data)
        ci_lower, ci_upper = np.percentile(model_data, [2.5, 97.5])  # 95% CI
        
        # Add median marker
        fig.add_trace(go.Scatter(
            x=[model],
            y=[median],
            mode='markers',
            marker=dict(color='black', size=15, symbol='circle'),
            showlegend=False
        ), row=row, col=col)
        
        # Add confidence interval
        fig.add_trace(go.Scatter(
            x=[model, model],
            y=[ci_lower, ci_upper],
            mode='lines',
            line=dict(color='black', width=2),
            showlegend=False
        ), row=row, col=col)

# Update layout
fig.update_layout(
    height=1000,  # Set the height of the plot
    width=1500,  # Set the width of the plot
    showlegend=True,
    template="simple_white",
    yaxis_title="Euclidean Distance (dva)"
)

# Ensure y-axis range is consistent for all subplots
fig.update_yaxes(range=[1, 7], row=1, col=1)
fig.update_yaxes(range=[1, 7], row=1, col=2)
fig.update_yaxes(range=[1, 7], row=2, col=1)
fig.update_yaxes(range=[1, 7], row=2, col=2)

fig.show()
#fig.write_image("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_calib/figures/group/ee_calib_plots.pdf")


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Box plot of the 75% percentile of EE for each subject for each run


# Create the figure
fig = go.Figure()

# Add a box plot for the mean percentiles
fig.add_trace(go.Box(
    y=pt_perc_task_all,
    name='PT Task all',
    boxmean=True,  # Display mean marker
    marker=dict(color='#5AC7DB')
))

# Add a box plot for the mean percentiles
fig.add_trace(go.Box(
    y=perc_task_all,
    name='FT Task all',
    boxmean=True,  # Display mean marker
    marker=dict(color='#069D6B')
))

# Add a box plot for the mean percentiles
fig.add_trace(go.Box(
    y=pt_perc_task_fix,
    name='PT Task fixation',
    boxmean=True,  # Display mean marker
    marker=dict(color='#5AC7DB')
))

# Add a box plot for the mean percentiles
fig.add_trace(go.Box(
    y=perc_task_fix,
    name='FT Task fixation',
    boxmean=True,  # Display mean marker
    marker=dict(color='#069D6B')
))

# Add a box plot for the mean percentiles
fig.add_trace(go.Box(
    y=pt_perc_task_pur,
    name='PT Task pursuit',
    boxmean=True,  # Display mean marker
    marker=dict(color='#5AC7DB')
))

# Add a box plot for the mean percentiles
fig.add_trace(go.Box(
    y=perc_task_pur,
    name='FT Task pursuit',
    boxmean=True,  # Display mean marker
    marker=dict(color='#069D6B')
))

# Add a box plot for the mean percentiles
fig.add_trace(go.Box(
    y=pt_perc_task_fv,
    name='PT Task freeview',
    boxmean=True,  # Display mean marker
    marker=dict(color='#5AC7DB')
))


# Add a box plot for the mean percentiles
fig.add_trace(go.Box(
    y=perc_task_fv,
    name='FT Task freeview',
    boxmean=True,  # Display mean marker
    marker=dict(color='#069D6B')
))

# Update layout
fig.update_layout(
    title="75th Percentiles Across Subjects with Confidence Intervals",
    yaxis_title="Euclidean Distance",
    showlegend=True,
    template="simple_white"
)

# Display the plot
fig.show()


In [63]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

# Prepare a list of dictionaries for the DataFrame
tasks = ['fixation', 'pursuit', 'freeview', 'all']

data_list = []
for subject in subjects:
    for task in tasks:
        data_list.append({
            'subject': subject,
            'task': task,
            'finetuned': subject_ee_means_ft[subject].get(task, None),
            'pretrained': subject_ee_means_pt[subject].get(task, None), 
            'scaled': subject_ee_means_sc[subject].get(task,None)
        })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data_list)

# Reshape data into long format for easier plotting
df_long = df.melt(id_vars=['subject', 'task'], value_vars=['pretrained', 'scaled', 'finetuned'], 
                   var_name='model', value_name='value')

# Define colors for each subject
colormap_subject_dict = {
    'sub-01': '#AA0DFE', 'sub-02': '#3283FE', 'sub-03': '#85660D', 'sub-04': '#782AB6',
    'sub-05': '#565656', 'sub-06': '#1C8356', 'sub-07': '#16FF32', 'sub-08': '#F7E1A0',
    'sub-09': '#E2E2E2', 'sub-11': '#1CBE4F', 'sub-13': '#DEA0FD', 'sub-14': '#FBE426', 'sub-15': '#325A9B'
}

# Create subplots with 2 rows and 2 columns
fig = make_subplots(
    rows=2, cols=2,  # Two rows, two columns
    subplot_titles=["<b>Guided Fixation task</b>", "<b>Pursuit task</b>", "<b>Freeview task</b>", "<b>All tasks</b>"],
    shared_yaxes=False, 
    shared_xaxes=False
)

# Add strip plots, subject connections, and median/confidence intervals for each task
for i, task in enumerate(tasks):
    row, col = divmod(i, 2)
    row += 1  # Adjusting for subplot indexing
    col += 1
    
    task_df = df_long[df_long['task'] == task]
    
    # Create a strip plot for the data distribution
    strip_fig = px.strip(task_df, x='model', y='value', color_discrete_sequence = ['gray'], stripmode='overlay')
    
    for trace in strip_fig.data:
        fig.add_trace(trace, row=row, col=col)
    
    for subject in task_df['subject'].unique():
        subject_data = task_df[task_df['subject'] == subject].set_index('model').loc[['pretrained', 'scaled', 'finetuned']].reset_index()
        color = colormap_subject_dict.get(subject, 'gray')
        
        # Add subject connecting lines
        fig.add_trace(go.Scatter(
            x=subject_data['model'],
            y=subject_data['value'],
            mode='lines',
            opacity=0.7,
            line=dict(color=color, width=0.8),  
            name=subject,
            legendgroup=subject,
            showlegend=False
        ), row=row, col=col)
    
    # Compute median and 95% confidence interval
    for model in ['pretrained', 'scaled', 'finetuned']:
        model_data = task_df[task_df['model'] == model]['value']
        median = np.median(model_data)
        ci_lower, ci_upper = np.percentile(model_data, [2.5, 97.5])  # 95% CI
        
        # Add median marker
        fig.add_trace(go.Scatter(
            x=[model],
            y=[median],
            mode='markers',
            marker=dict(color='black', size=15, symbol='circle'),
            showlegend=False
        ), row=row, col=col)
        
        # Add confidence interval
        fig.add_trace(go.Scatter(
            x=[model, model],
            y=[ci_lower, ci_upper],
            mode='lines',
            line=dict(color='black', width=2),
            showlegend=False
        ), row=row, col=col)

# Update layout
fig.update_layout(
    height=1000,  # Set the height of the plot
    width=1500,  # Set the width of the plot
    showlegend=True,
    template="simple_white",
    font=dict(size=15),
    yaxis_title="Euclidean Distance (dva)"
)

# Ensure y-axis range is consistent for all subplots
fig.update_yaxes(range=[1, 7], row=1, col=1)
fig.update_yaxes(range=[1, 7], row=1, col=2)
fig.update_yaxes(range=[1, 7], row=2, col=1)
fig.update_yaxes(range=[1, 7], row=2, col=2)

fig.show()
#fig.write_image("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_calib/figures/group/ee_calib_plots.pdf")


In [36]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd
import numpy as np
from plotly.subplots import make_subplots

# Prepare a list of dictionaries for the DataFrame
tasks = ['fixation', 'pursuit', 'freeview', 'all']

data_list = []
for subject in subjects:
    for task in tasks:
        data_list.append({
            'subject': subject,
            'task': task,
            'original': subject_ee_means_ft_orig[subject].get(task, None),
            'no_interpol': subject_ee_means_ft[subject].get(task, None)
        })

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data_list)

# Reshape data into long format for easier plotting
df_long = df.melt(id_vars=['subject', 'task'], value_vars=['original', 'no_interpol'], 
                   var_name='model', value_name='value')

# Define colors for each subject
colormap_subject_dict = {
    'sub-01': '#AA0DFE', 'sub-02': '#3283FE', 'sub-03': '#85660D', 'sub-04': '#782AB6',
    'sub-05': '#565656', 'sub-06': '#1C8356', 'sub-07': '#16FF32', 'sub-08': '#F7E1A0',
    'sub-09': '#E2E2E2', 'sub-11': '#1CBE4F', 'sub-13': '#DEA0FD', 'sub-14': '#FBE426', 'sub-15': '#325A9B'
}

# Create subplots with 2 rows and 2 columns
fig = make_subplots(
    rows=2, cols=2,  # Two rows, two columns
    subplot_titles=["<b>Guided Fixation task</b>", "<b>Pursuit task</b>", "<b>Freeview task</b>", "<b>All tasks</b>"],
    shared_yaxes=False, 
    shared_xaxes=False
)

# Add strip plots, subject connections, and median/confidence intervals for each task
for i, task in enumerate(tasks):
    row, col = divmod(i, 2)
    row += 1  # Adjusting for subplot indexing
    col += 1
    
    task_df = df_long[df_long['task'] == task]
    
    # Create a strip plot for the data distribution
    strip_fig = px.strip(task_df, x='model', y='value', color_discrete_sequence = ['gray'], stripmode='overlay')
    
    for trace in strip_fig.data:
        fig.add_trace(trace, row=row, col=col)
    
    for subject in task_df['subject'].unique():
        subject_data = task_df[task_df['subject'] == subject].set_index('model').loc[['original', 'no_interpol']].reset_index()
        color = colormap_subject_dict.get(subject, 'gray')
        
        # Add subject connecting lines
        fig.add_trace(go.Scatter(
            x=subject_data['model'],
            y=subject_data['value'],
            mode='lines',
            opacity=0.7,
            line=dict(color=color, width=0.8),  
            name=subject,
            legendgroup=subject,
            showlegend=True
        ), row=row, col=col)
    
    # Compute median and 95% confidence interval
    for model in ['original', 'no_interpol']:
        model_data = task_df[task_df['model'] == model]['value']
        median = np.median(model_data)
        ci_lower, ci_upper = np.percentile(model_data, [2.5, 97.5])  # 95% CI
        
        # Add median marker
        fig.add_trace(go.Scatter(
            x=[model],
            y=[median],
            mode='markers',
            marker=dict(color='black', size=15, symbol='circle'),
            showlegend=False
        ), row=row, col=col)
        
        # Add confidence interval
        fig.add_trace(go.Scatter(
            x=[model, model],
            y=[ci_lower, ci_upper],
            mode='lines',
            line=dict(color='black', width=2),
            showlegend=False
        ), row=row, col=col)

# Update layout
fig.update_layout(
    height=1000,  # Set the height of the plot
    width=1500,  # Set the width of the plot
    showlegend=True,
    template="simple_white",
    font=dict(size=15),
    yaxis_title="Euclidean Distance (dva)"
)

# Ensure y-axis range is consistent for all subplots
fig.update_yaxes(range=[1, 6], row=1, col=1)
fig.update_yaxes(range=[1, 6], row=1, col=2)
fig.update_yaxes(range=[1, 6], row=2, col=1)
fig.update_yaxes(range=[1, 6], row=2, col=2)

fig.show()
#fig.write_image("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_calib/figures/group/new_labels_comparisons.pdf")


In [ ]:
import pandas as pd
from scipy.signal import resample

design_matrix = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_calib/exp_design/sub-09/sub-09_ses-02_task-DeepMReyeCalib_run-01_design_matrix.tsv", sep ="\t")
task_1_trials = np.array(design_matrix["task_fixation"])
task_2_trials = np.array(design_matrix["task_pursuit"])
task_3_trials = np.array(design_matrix["task_freeviewing"])

resampled_task_1_trials = resample(task_1_trials, 154)
resampled_task_2_trials = resample(task_2_trials, 154)
resampled_task_3_trials = resample(task_3_trials, 154)

task_1_trials_bool = resampled_task_1_trials > 0.5
task_2_trials_bool = resampled_task_2_trials > 0.5
task_3_trials_bool = resampled_task_3_trials > 0.5

print(len(task_1_trials_bool))

154


In [96]:
# Load model predictions
evaluation_ft = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_calib/pred/evaluation_dict_calib_no_interpol.npy", allow_pickle=True).item()
evaluation_ft_orig = np.load("/Users/sinakling/disks/meso_shared/deepmreye/derivatives/deepmreye_calib/pred/evaluation_dict_calib.npy", allow_pickle=True).item()

subject_data_ft = evaluation_ft[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_calib/pp_data_no_interpol/sub-09_DeepMReyeCalib_label_no_interpol.npz"]
df_pred_median_ft, df_pred_subtr_ft = adapt_evaluation(subject_data_ft)

subject_data_ft_orig = evaluation_ft_orig[f"/scratch/mszinte/data/deepmreye/derivatives/deepmreye_calib/pp_data/sub-09_DeepMReyeCalib_label.npz"]
df_pred_median_ft_orig, df_pred_subtr_ft_orig = adapt_evaluation(subject_data_ft_orig)
#sub_run_X_ft = np.array(df_pred_median_ft['X'][:154])
#sub_run_Y_ft = np.array(df_pred_median_ft['Y'][:154])
#sub_run_X_ft = sub_run_X_ft[task_2_trials_bool]   
#sub_run_Y_ft = sub_run_Y_ft[task_2_trials_bool] 

eye_data = pd.read_csv(f"/Users/sinakling/disks/meso_shared/deepmreye/derivatives/pp_data/sub-09/eyetracking/sub-09_task-DeepMReyeCalib_run_01_eyedata.tsv.gz", compression='gzip', delimiter='\t')
eye_data = eye_data[['timestamps','x', 'y']].to_numpy()     
# downsample eye data 
#eye_data_downsampled = downsample_to_targetrate(eye_data, 1000, 8.33)
#eye_data_downsampled = eye_data_downsampled[:1540,:]
eye_data_downsampled_x = chunk_and_median(eye_data[:,1])
eye_data_downsampled_y = chunk_and_median(eye_data[:,2])
eye_data_downsampled = np.stack((eye_data_downsampled_x, eye_data_downsampled_y), axis=1)

#eye_data_downsampled_task_2 = eye_data_downsampled[task_2_trials_bool] 



In [98]:
fig = go.Figure()

fig.add_trace(go.Scatter(y=eye_data_downsampled[107:,0],
                    mode='lines', name='Eyetracking', line=dict(color='#0E1C36', width=2)))
fig.add_trace(go.Scatter(y=np.array(df_pred_median_ft_orig['X'])[:154][107:],
                    mode='lines', name='FT original', line=dict(color='#66E2B8', width=2)))
fig.add_trace(go.Scatter(y=np.array(df_pred_median_ft['X'])[:154][107:],
                    mode='lines', name='FT no interpol', line=dict(color='#069D6B', width=2)))

# Update layout
fig.update_layout(
    showlegend=True,
    template="simple_white",
    font=dict(size=13),
    yaxis_title="Euclidean Distance (dva)"
)
fig.update_yaxes(range=[-10, 13])

fig.show()